#**Limpieza y Procesamiento de Aplicaciones de Google Play Store**


## Sobre el conjunto de datos

El conjunto de datos **"Google Play Store Apps"** en Kaggle contiene información recopilada de más de 10,000 aplicaciones disponibles en Google Play Store. Este dataset fue creado mediante web scraping y está diseñado para facilitar el análisis del mercado de aplicaciones Android.

Incluye diversas características de cada aplicación, como el **nombre, la categoría, la calificación, el número de reseñas, el tamaño de la aplicación, el número de instalaciones, el tipo (gratuita o de pago), el precio, la clasificación de contenido, la última fecha de actualización, la versión actual y la versión mínima de Android requerida.**

[Consulte el link para acceso a los datos](https://www.kaggle.com/datasets/lava18/google-play-store-apps)







## Configuracion del entorno de trabajo

En esta sección, configuramos el entorno de PySpark y cargamos los datos. PySpark es una API de Spark para Python que nos permite trabajar con grandes volúmenes de datos de manera distribuida.


*   **SparkSession:** Es el punto de entrada para trabajar con Spark. Nos permite crear DataFrames y ejecutar operaciones.

*   **Cargar datos:** Usamos spark.read.csv para cargar los archivos CSV en DataFrames.


In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, regexp_replace, when, split, expr, isnull, count
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, VectorAssembler, Imputer

# Crear sesión de Spark
spark = SparkSession.builder.appName("Limpieza y Procesamiento de Datos en PySpark").getOrCreate()

## Carga de datos

In [3]:
# Cargar el dataset
df = spark.read.csv("./googleplaystore.csv", header=True, inferSchema=True)

# Mostrar esquema original
df.printSchema()

# Mostrar las primeras filas
df.show(5)

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: string (nullable = true)
 |-- Reviews: string (nullable = true)
 |-- Size: string (nullable = true)
 |-- Installs: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- Content Rating: string (nullable = true)
 |-- Genres: string (nullable = true)
 |-- Last Updated: string (nullable = true)
 |-- Current Ver: string (nullable = true)
 |-- Android Ver: string (nullable = true)

+--------------------+--------------+------+-------+----+-----------+----+-----+--------------+--------------------+----------------+------------------+------------+
|                 App|      Category|Rating|Reviews|Size|   Installs|Type|Price|Content Rating|              Genres|    Last Updated|       Current Ver| Android Ver|
+--------------------+--------------+------+-------+----+-----------+----+-----+--------------+--------------------+----------------+--------------

## Limpieza, transformacion de datos

In [4]:
# Número de filas en el DataFrame
num_filas = df.count()

print(f"El número de filas en el dataset es: {num_filas}")

El número de filas en el dataset es: 10841


Las columnas son de tipo string, lo que significa que los valores NaN no se están manejando correctamente, ya que NaN es un valor numérico y no se aplica a columnas de tipo string. Además, los valores NULL en columnas de tipo string pueden confundirse con cadenas vacías ("") o cadenas como "NULL" o "NaN"

Por lo tanto convertimos las columnas a un tipo de dato (segun corresponda) que no sea string

In [5]:

from pyspark.sql import functions as F

# Convertimoos 'Size' a un valor numérico en megabytes
df = df.withColumn(
    "Size_MB",
    F.when(F.col("Size").endswith("M"), F.regexp_extract(F.col("Size"), r"(\d+\.?\d*)", 1).cast("float"))
    .when(F.col("Size").endswith("K"), F.regexp_extract(F.col("Size"), r"(\d+\.?\d*)", 1).cast("float") / 1024)
    .when(F.col("Size").endswith("k"), F.regexp_extract(F.col("Size"), r"(\d+\.?\d*)", 1).cast("float") / 1024)
)

df = df.drop("Size")

In [6]:
from pyspark.sql.functions import col

# Convertimos columnas numéricas
df = df.withColumn("Rating", col("Rating").cast("double")) \
       .withColumn("Reviews", col("Reviews").cast("int")) \
       .withColumn("Price", col("Price").cast("double"))

In [7]:
from pyspark.sql.functions import when

# Reemplazamos  valores no estándar por NULL (Al ser string pyspark puede interpretar un NULL como cadena)

for column in df.columns:
    df = df.withColumn(column, when((col(column) == "") | (col(column).rlike("^\s*$")) | (col(column) == "NULL") | (col(column) == "NaN"), None).otherwise(col(column)))

<>:6: SyntaxWarning: invalid escape sequence '\s'
<>:6: SyntaxWarning: invalid escape sequence '\s'
C:\Users\ENRIQUE\AppData\Local\Temp\ipykernel_32672\3963064863.py:6: SyntaxWarning: invalid escape sequence '\s'
  df = df.withColumn(column, when((col(column) == "") | (col(column).rlike("^\s*$")) | (col(column) == "NULL") | (col(column) == "NaN"), None).otherwise(col(column)))


In [8]:
from pyspark.sql.functions import col, sum, isnan

# Contamos  NULL y NaN por columna
nan_null_counts = df.select([
    (
        sum(col(column).isNull().cast("int")) +  # Conteo de NULL
        sum(isnan(col(column)).cast("int"))      # Conteo de NaN
    ).alias(column)
    for column in df.columns
])

nan_null_counts.show()

+---+--------+------+-------+--------+----+-----+--------------+------+------------+-----------+-----------+-------+
|App|Category|Rating|Reviews|Installs|Type|Price|Content Rating|Genres|Last Updated|Current Ver|Android Ver|Size_MB|
+---+--------+------+-------+--------+----+-----+--------------+------+------------+-----------+-----------+-------+
|  0|       0|  1476|      3|       0|   1|  803|             1|     0|           0|          8|          3|   1697|
+---+--------+------+-------+--------+----+-----+--------------+------+------------+-----------+-----------+-------+



In [9]:
# Eliminar los registros nulos
# En este caso eliminamos lo registros con valores nulo, esto porque los valores nulos por columna no representan mas del 50%

df_clean = df.dropna(how='any')

In [10]:
df_clean.show(5)

+--------------------+--------------+------+-------+-----------+----+-----+--------------+--------------------+----------------+------------------+------------+-----------------+
|                 App|      Category|Rating|Reviews|   Installs|Type|Price|Content Rating|              Genres|    Last Updated|       Current Ver| Android Ver|          Size_MB|
+--------------------+--------------+------+-------+-----------+----+-----+--------------+--------------------+----------------+------------------+------------+-----------------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159|    10,000+|Free|  0.0|      Everyone|        Art & Design| January 7, 2018|             1.0.0|4.0.3 and up|             19.0|
| Coloring book moana|ART_AND_DESIGN|   3.9|    967|   500,000+|Free|  0.0|      Everyone|Art & Design;Pret...|January 15, 2018|             2.0.0|4.0.3 and up|             14.0|
|U Launcher Lite –...|ART_AND_DESIGN|   4.7|  87510| 5,000,000+|Free|  0.0|      Everyone|        Art & D

In [11]:
# Contamos el numero de registros hasta el momento

df_clean.count()

7145

In [12]:
# Indntificamos valores nulos por columna

nan_null_counts = df_clean.select([
    (
        sum(col(column).isNull().cast("int")) +  # Conteo de NULL
        sum(isnan(col(column)).cast("int"))      # Conteo de NaN
    ).alias(column)  # Asignar el nombre de la columna original
    for column in df_clean.columns
])


nan_null_counts.show()

+---+--------+------+-------+--------+----+-----+--------------+------+------------+-----------+-----------+-------+
|App|Category|Rating|Reviews|Installs|Type|Price|Content Rating|Genres|Last Updated|Current Ver|Android Ver|Size_MB|
+---+--------+------+-------+--------+----+-----+--------------+------+------------+-----------+-----------+-------+
|  0|       0|     0|      0|       0|   0|    0|             0|     0|           0|          0|          0|      0|
+---+--------+------+-------+--------+----+-----+--------------+------+------------+-----------+-----------+-------+



In [13]:
#  Eliminamos datos duplicados en base a la columna "App"

df_clean = df_clean.dropDuplicates(["App"])

In [14]:
df_clean.count()

6484

#### Identificamos valores unicos en columnas con texto para normalizar

In [ ]:
valores_unicos = df_clean.select("Category").distinct()

# Mostrar los valores únicos
valores_unicos.show()

+-------------------+
|           Category|
+-------------------+
|             EVENTS|
|             COMICS|
|             SPORTS|
|            WEATHER|
|      VIDEO_PLAYERS|
|  AUTO_AND_VEHICLES|
|          PARENTING|
|      ENTERTAINMENT|
|    PERSONALIZATION|
| HEALTH_AND_FITNESS|
|   TRAVEL_AND_LOCAL|
|BOOKS_AND_REFERENCE|
|     FOOD_AND_DRINK|
|        PHOTOGRAPHY|
|           BUSINESS|
|             FAMILY|
|           SHOPPING|
|     HOUSE_AND_HOME|
|               GAME|
|          EDUCATION|
+-------------------+
only showing top 20 rows



In [15]:
# El mismo objetivo que la celda anterior,ahora sobre todas la columnas

list_var = ["Installs", "Type", "Price", "Content Rating", "Genres", "Current Ver", "Last Updated"]


# Iteraramos sobre cada columna en la lista
for columna in list_var:
    print(f"Valores únicos para la columna: {columna}")
    valores_unicos = df_clean.select(columna).distinct()
    valores_unicos.show()

Valores únicos para la columna: Installs
+--------------+
|      Installs|
+--------------+
|       50,000+|
|          100+|
|    5,000,000+|
|  100,000,000+|
|1,000,000,000+|
|    1,000,000+|
|  500,000,000+|
|           10+|
|           50+|
|            5+|
|          500+|
|        1,000+|
|      500,000+|
|        5,000+|
|   10,000,000+|
|   50,000,000+|
|      100,000+|
|            1+|
|       10,000+|
+--------------+

Valores únicos para la columna: Type
+----+
|Type|
+----+
|Free|
+----+

Valores únicos para la columna: Price
+-----+
|Price|
+-----+
|  0.0|
+-----+

Valores únicos para la columna: Content Rating
+---------------+
| Content Rating|
+---------------+
|        Unrated|
|           Teen|
|     Mature 17+|
|   Everyone 10+|
|       Everyone|
|Adults only 18+|
+---------------+

Valores únicos para la columna: Genres
+--------------------+
|              Genres|
+--------------------+
|Video Players & E...|
|Adventure;Action ...|
|           Education|
|         

In [16]:
# Limpiar y convertir a enteros

df_clean = df_clean.withColumn("Installs", regexp_replace(col("Installs"), "[+,]", "").cast("int"))

In [17]:
df_clean.show(5)

+--------------------+---------------+------+-------+--------+----+-----+--------------+---------------+------------------+-----------+------------+-----------------+
|                 App|       Category|Rating|Reviews|Installs|Type|Price|Content Rating|         Genres|      Last Updated|Current Ver| Android Ver|          Size_MB|
+--------------------+---------------+------+-------+--------+----+-----+--------------+---------------+------------------+-----------+------------+-----------------+
|"Alphabet ""H"" P...|PERSONALIZATION|   4.5|      2|     100|Free|  0.0|      Everyone|Personalization| December 21, 2017|          1|  4.1 and up|              3.0|
|"Eat Fast Prepare...| FOOD_AND_DRINK|   4.6|   4925| 1000000|Free|  0.0|      Everyone|   Food & Drink|     June 10, 2018|      3.6.6|4.0.3 and up|             17.0|
|+Download 4 Insta...|         SOCIAL|   4.5|  40467| 1000000|Free|  0.0|      Everyone|         Social|    August 2, 2018|       5.03|  4.1 and up|             22.0

## Actividad en clase

Hasta este momento tenemos un dataset **"mas o menos"** limpio. ¿Con el dataset hasta este momento, puedes emplear un modelo de ML?




 1. **Existen columnas que no aportan información. Identifica cuáles son y explica brevemente por qué deberías eliminarlas del dataset**

2. **En la columna "Last Update" la fecha esta en un formato diferente, tranformala en un valor numerico (diferencia del año actual con el año en cuestion)**

**Pistas:**

Tranformar fecha a un formato especifico:


```
df.withColumn("Date_Update", to_date(col("Fecha"), "MMMM d, yyyy"))

```

Diferencia de años



```
df.withColumn("Diferencia_Anios", year(current_date()) - year(col("Date_Update")))
```




In [ ]:
df_clean.withColumn("Date_Update", to_date(col("Fecha"), "MMMM d, yyyy"))

DataFrame[App: string, Category: string, Rating: double, Reviews: int, Installs: int, Type: string, Price: double, Content Rating: string, Genres: string, Last Updated: string, Current Ver: string, Android Ver: string, Size_MB: double]

3. **Category, Genres y Content Rating: Convertir cada categoría en un número único asignando un índice a cada una.**

(por ejemplo, "Everyone" → 0, "Teen" → 1, "Mature 17+" → 2, etc.)

**Pista:**


```
df.withColumn(
    "Tipos de auto",
    when(col("Tipos de auto") == "Combustible (Gasolina)", 0)
    .when(col("Tipos de auto") == "Electrico", 1)
    .when(col("Tipos de auto") == "Hibrido", 2)
    .otherwise(-1)
)

```

